In this assignment, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:

Run LASSO with different L1 penalties.
Choose best L1 penalty using a validation set.
Choose best L1 penalty using a validation set, with additional constraint on the size of subset.


In [1]:
import pandas as pd
import numpy as np
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

Create new features by performing following transformation on inputs:

In [2]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.


Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

In [3]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

In [4]:
pd.Series(model_all.coef_,index=all_features)

bedrooms                0.000000
bedrooms_square         0.000000
bathrooms               0.000000
sqft_living           134.439314
sqft_living_sqrt        0.000000
sqft_lot                0.000000
sqft_lot_sqrt           0.000000
floors                  0.000000
floors_square           0.000000
waterfront              0.000000
view                24750.004586
condition               0.000000
grade               61749.103091
sqft_above              0.000000
sqft_basement           0.000000
yr_built               -0.000000
yr_renovated            0.000000
dtype: float64

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [5]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [6]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [7]:
l1_penalty = np.logspace(1, 7, num=13)
l1_penalty

array([  1.00000000e+01,   3.16227766e+01,   1.00000000e+02,
         3.16227766e+02,   1.00000000e+03,   3.16227766e+03,
         1.00000000e+04,   3.16227766e+04,   1.00000000e+05,
         3.16227766e+05,   1.00000000e+06,   3.16227766e+06,
         1.00000000e+07])

Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:

Compute the RSS on VALIDATION for the current model (print or save the RSS)

Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?



In [8]:
for i in l1_penalty:
    train_model_all = linear_model.Lasso(i, normalize=True) # set parameters
    train_model_all.fit(training[all_features], training['price']) # learn weights
    y = pd.Series(train_model_all.coef_, index=all_features)
    RSS = np.sum((train_model_all.predict(validation[all_features])-validation['price'])**2)
    print i,RSS

10.0 3.982133273e+14
31.6227766017 3.99041900253e+14
100.0 4.29791604073e+14
316.227766017 4.63739831045e+14
1000.0 6.45898733634e+14
3162.27766017 1.22250685943e+15
10000.0 1.22250685943e+15
31622.7766017 1.22250685943e+15
100000.0 1.22250685943e+15
316227.766017 1.22250685943e+15
1000000.0 1.22250685943e+15
3162277.66017 1.22250685943e+15
10000000.0 1.22250685943e+15


Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [9]:
train2_model_all = linear_model.Lasso(alpha=10.0, normalize=True)

In [10]:
train2_model_all.fit(training[all_features], training['price'])

Lasso(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [11]:
np.count_nonzero(train2_model_all.coef_) + np.count_nonzero(train2_model_all.intercept_)

15

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

Assign 7 to the variable ‘max_nonzeros’.

Exploring large range of l1_penalty

For l1_penalty in np.logspace(1, 4, num=20):

Quiz Question: What values did you find for l1_penalty_min and l1_penalty_max?

In [12]:
max_nonzeros = 7

In [13]:
alpha = np.logspace(1, 4, num=20)

In [14]:
counts_of_nonzeros = []
for l1_penalty in alpha:
    train_model_2 = linear_model.Lasso(l1_penalty, normalize=True) # set parameters
    train_model_2.fit(training[all_features], training['price']) # learn weights
    if train_model_2.intercept_ <> 0:
        counts = np.count_nonzero(train_model_2.coef_) + np.count_nonzero(train_model_2.intercept_)
        counts_of_nonzeros.append((l1_penalty,counts))
    elif train_model_2.intercept_==0:
        counts = np.count_nonzero(train_model_2.coef_) + 1
        counts_of_nonzeros.append((l1_penalty,counts))
    
        
        

In [15]:
print counts_of_nonzeros

[(10.0, 15), (14.384498882876629, 15), (20.691380811147901, 15), (29.763514416313178, 15), (42.813323987193932, 13), (61.584821106602639, 12), (88.586679041008225, 11), (127.42749857031335, 10), (183.29807108324357, 7), (263.66508987303581, 6), (379.26901907322497, 6), (545.55947811685144, 6), (784.75997035146065, 5), (1128.8378916846884, 3), (1623.776739188721, 3), (2335.7214690901214, 2), (3359.8182862837812, 1), (4832.9302385717519, 1), (6951.9279617756056, 1), (10000.0, 1)]


In [16]:
list_greater = [k for k,v in counts_of_nonzeros if v > 7]

In [17]:
l1_penalty_min = max(list_greater)

In [18]:
l1_penalty_min

127.42749857031335

In [19]:
list_less = [m for m,n in counts_of_nonzeros if n < 7]
l1_penalty_max = min(list_less)
l1_penalty_max

263.66508987303581

Exploring narrower range of l1_penalty

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [20]:
counts_of_nonzeros_2 = []
RSS_list = []
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    train_model_3 = linear_model.Lasso(l1_penalty, normalize=True) # set parameters
    train_model_3.fit(training[all_features], training['price']) # learn weights
    RSS = np.sum((train_model_3.predict(validation[all_features])-validation['price'])**2)
    RSS_list.append((l1_penalty,RSS))
    if train_model_3.intercept_ <> 0:
        counts = np.count_nonzero(train_model_3.coef_) + np.count_nonzero(train_model_3.intercept_)
        counts_of_nonzeros_2.append((l1_penalty,counts))
    elif train_model_3.intercept_==0:
        counts = np.count_nonzero(train_model_3.coef_) + 1
        counts_of_nonzeros_2.append((l1_penalty,counts))
    

In [21]:
max_7_list = [j for j,p in counts_of_nonzeros_2 if p == 7] 
max_7_list

[156.10909673930755,
 163.27949628155611,
 170.44989582380464,
 177.6202953660532,
 184.79069490830176,
 191.96109445055032,
 199.13149399279888]

In [22]:
RSS_list_2 = [(y,x) for x,y in RSS_list if x in max_7_list]

In [23]:
RSS_list_2.sort()

In [24]:
RSS_list_2

[(440037365263316.94, 156.10909673930755),
 (440777489641605.56, 163.27949628155611),
 (441566698090138.94, 170.44989582380464),
 (442406413188665.06, 177.6202953660532),
 (443296716874313.06, 184.79069490830176),
 (444239780526141.2, 191.96109445055032),
 (445230739842613.8, 199.13149399279888)]

In [25]:
train_model_4 = linear_model.Lasso(max_7_list[0], normalize=True) # set parameters
train_model_4.fit(training[all_features], training['price']) # learn weights

Lasso(alpha=156.10909673930755, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=True, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [26]:
pd.Series(train_model_4.coef_, index = all_features)

bedrooms                -0.000000
bedrooms_square         -0.000000
bathrooms            10610.890284
sqft_living            163.380252
sqft_living_sqrt         0.000000
sqft_lot                -0.000000
sqft_lot_sqrt           -0.000000
floors                   0.000000
floors_square            0.000000
waterfront          506451.687115
view                 41960.043555
condition                0.000000
grade               116253.553700
sqft_above               0.000000
sqft_basement            0.000000
yr_built             -2612.234880
yr_renovated             0.000000
dtype: float64